### Installation des dépendances

In [ ]:
%pip install numpy
%pip install opencv-python
%pip install -U matplotlib
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
import math
import os
import time

In [187]:
# Define Constants
STEP_TO_NEXT_FRAME = 4 
""" in each video, we will read 1 frame every 24/STEP_TO_INDEX frames"""
DEBUG = False # if True, print debug information
NUMBER_OF_VIDEO = 100 # number of videos to index
MAX_FRAME_INDEXATION_TIME = 3 # in seconds
MAX_SEARCH_TIME = 0.1 # in seconds
VIDEOS_SIZE_ON_DISK = 444039168  # in octets
MAX_FRAME_NUMBER = 37895 # maximum number of frames in a video
N = int(MAX_FRAME_NUMBER/STEP_TO_NEXT_FRAME) # index Length
D = 8 # index Depth

In [188]:
#Define all global variables
imageList = []
videoList = []
# SearchResult = [] # List of (image name, video name, time)
indexationTable = [] # Table of indexation
Descriptor_list = np.empty((N, D * D * D), dtype=np.float32) # Descriptor of the image

### Lecture des fichiers

In [189]:
imagePath = "/data/jpeg/"
videoPath = "/data/mp4/"
currDirectory = os.getcwd()
imageList = os.listdir(currDirectory + imagePath)
videoList = os.listdir(currDirectory + videoPath)

In [190]:
def getVideoParameter(videoPath:str):
    """return the frame rate and the number of frames of the video."""
    cap = cv.VideoCapture(videoPath)
    fps = cap.get(cv.CAP_PROP_FPS)
    frame_count = cap.get(cv.CAP_PROP_FRAME_COUNT)
    return fps, frame_count
    

In [191]:
def displayRGBHistogram(histogram_r, histogram_g, histogram_b):
    plt.plot(histogram_b, color='blue', label='Canal bleu')
    plt.plot(histogram_g, color='green', label='Canal vert')
    plt.plot(histogram_r, color='red', label='Canal rouge')
    plt.xlabel('Intensité des pixels')
    plt.ylabel('Nombre de pixels')
    plt.title('Histogramme de l\'image')
    plt.legend()
    plt.show()

In [192]:

def convertImageToVector(imagePath, needToRead:bool = True):
    """return the histogram of the image."""
    if needToRead:
        image = cv.imread(imagePath)
    else:
        image = imagePath

    image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
    histogram = cv.calcHist([image], [0, 1, 2], None, [D, D, D], [0, 256, 0, 256, 0, 256])
    histogram = cv.normalize(histogram, histogram).flatten()
    return histogram


In [193]:
def createIndexTable():
    """Index a video by creating a list of frames and their corresponding index. and update the indexationTable variable."""
    for video in videoList:
        path = currDirectory + videoPath + video
        fps, frame_count= getVideoParameter(path)
        cap = cv.VideoCapture(path)
        startIndex = len(indexationTable)
        stopIndex = startIndex - 1 + math.floor(frame_count/STEP_TO_NEXT_FRAME)
        for i in range(startIndex, stopIndex):
            frameNumber = STEP_TO_NEXT_FRAME*(i-startIndex)
            indexationTable.append((i, video, (frameNumber/fps)))
            currImage = cap.set(cv.CAP_PROP_POS_FRAMES, frameNumber)
            ret, currImage = cap.read()
            Descriptor_list[i] = convertImageToVector(currImage, False)
        
createIndexTable()

In [194]:
def CalculateCompressionRate():
    rate = 1- (3*D*N*4)/VIDEOS_SIZE_ON_DISK #*4 because we use float32 and 3 because we have 3 channels RGB
    print("Le taux de compression est: ",rate)
    
CalculateCompressionRate()

Le taux de compression est:  0.9979519644537304


In [195]:
def plotImage(image, type):
    figure = plt.figure(figsize = (10,10))
    imageout = np.clip(image,0,255)
    imageout= imageout.astype(type)
    plt.imshow(imageout, cmap='gray')
    plt.show()

In [196]:
def searchImage():
    SearchResult = []
    for image in imageList:
        path = currDirectory + imagePath + image
        if DEBUG:
            plotImage(cv.cvtColor(cv.imread(path), cv.COLOR_BGR2RGB), 'uint8')
        imageVector = convertImageToVector(path)
        for i in range(len(indexationTable)):
            dist = cv.compareHist(imageVector, Descriptor_list[i], cv.HISTCMP_BHATTACHARYYA)
            # dist = np.linalg.norm(imageVector-Descriptor_list[i])
            if i == 0:
                bestDistance = dist
                bestIndex = i
            if dist < bestDistance:
                bestDistance = dist
                bestIndex = i

        if bestDistance < 0.346:
            bestVideo = indexationTable[bestIndex][1]
            bestTime = indexationTable[bestIndex][2]
        else:
            bestVideo = "out"
            bestTime = ""
        
        # print("the best distance is", bestDistance)
        
        # print("The best match is the frame number ", (image.split(".")[0], bestVideo.split(".")[0], bestTime)) 
        if type(bestTime) == float: 
            SearchResult.append((image.split(".")[0], bestVideo.split(".")[0], round(bestTime, 3)))
        else:
            SearchResult.append((image.split(".")[0], bestVideo.split(".")[0], bestTime)) 
        # break
    return SearchResult

SearchResult = searchImage()
    


In [197]:
def writeResultInCSVFile():
    """Function to write the results in a csv file."""
    with open(currDirectory+'/src/test.csv', 'w') as file:
        file.write("image,video_pred,minutage_pred\n")
        for imageName, video, time in SearchResult:
            file.write(imageName + "," + video + "," + str(time) + "\n")
    file.close()

writeResultInCSVFile()

In [198]:
# Affichage des résultats
import subprocess
directory = r'C:\Users\loicn\OneDrive - polymtl.ca\Polymtl\Session_5\INF8770_TEC_MULTIMEDIA\Lab\Lab3\src'
output = subprocess.run(['python3', 'evaluate.py', '--file', 'test.csv', '--file_gt', 'gt.csv'], cwd=directory, capture_output=True, text=True)

print(output.stdout)

Taux de bonnes rÃ©ponses : 90.6% (906/1000)
Ecart temporel moyen : 1.48 sec

